# Milestone 1: Data Project Proposal and ETL

**Course Project Website Draft**

## Team
Solo project (no partner).

## Project Direction (1-3 Candidate Datasets)
For this project, I want to build a public, end-to-end walkthrough that starts with data extraction, continues through cleaning and exploratory analysis, and ends with a clear managerial insight. I am currently considering three realistic datasets and will finalize one after deeper quality checks.

**Candidate 1: Retail orders and returns data (CSV export format)**  
Why it is promising: this type of data usually includes transaction date, product category, quantity, sales amount, and return status. That structure is good for ETL because it requires date parsing, missing value checks, and joining transactions with returns. It is also useful for business decisions around inventory and pricing.  
Question to answer: *Which product categories drive the highest revenue after accounting for returns, and how should managers adjust inventory priorities by month?*

**Candidate 2: City bike-share trip data (public transportation logs)**  
Why it is promising: trip records often include start station, end station, start time, duration, and rider type. This allows time-based demand analysis and station-level congestion patterns. ETL challenges include filtering abnormal trip durations and engineering features like day-of-week and hour-of-day.  
Question to answer: *What are the peak demand windows by station cluster, and where should operators rebalance bikes to reduce stockouts?*

**Candidate 3: Restaurant inspection records (city open data portal)**  
Why it is promising: inspection data includes violation category, score, inspection date, and neighborhood. It supports operational insights for quality management and compliance risk monitoring. ETL involves standardizing violation labels, handling repeated inspections per location, and creating interpretable risk indicators.  
Question to answer: *Which violation categories most strongly predict repeat low scores, and where should inspectors focus follow-up visits?*

At this stage, Candidate 1 is my likely final choice because the path from raw data to decision is straightforward and aligns with a managerial audience: category performance, return-adjusted revenue, and seasonal planning.

## Collaboration Plan
Since I am working solo, I will use a structured weekly workflow to maintain accountability: (1) one coding block for extraction and cleaning, (2) one analysis block for charts and metrics, and (3) one documentation block for notebook updates and GitHub commits. I am using Git + GitHub for version control, Jupyter Notebook for analysis, and GitHub Pages for publication. I will commit incremental progress after each major ETL or analysis change and keep the repository history readable with clear commit messages.

## ETL Demonstration (Draft)
Below is a small synthetic retail sample showing the ETL workflow I plan to apply to the final dataset: parse dates, normalize categories, compute net sales, and create a tidy table suitable for analysis. This is not the final data source, but it demonstrates that the extraction and transformation pipeline works end-to-end.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style='whitegrid')

In [ ]:
# Extract (sample raw data to mimic CSV ingestion)
raw_data = [
    {'order_id': 1001, 'order_date': '2025-09-01', 'region': 'North', 'product': 'Office-Chair', 'quantity': 2, 'unit_price': 120, 'returned': 0},
    {'order_id': 1002, 'order_date': '2025-09-03', 'region': 'South', 'product': 'Office-Desk', 'quantity': 1, 'unit_price': 260, 'returned': 0},
    {'order_id': 1003, 'order_date': '2025-09-05', 'region': 'West',  'product': 'Tech-Headset', 'quantity': 3, 'unit_price': 40,  'returned': 1},
    {'order_id': 1004, 'order_date': '2025-10-01', 'region': 'North', 'product': 'Office-Chair', 'quantity': 1, 'unit_price': 120, 'returned': 0},
    {'order_id': 1005, 'order_date': '2025-10-08', 'region': 'East',  'product': 'Tech-Mouse',   'quantity': 5, 'unit_price': 25,  'returned': 0},
    {'order_id': 1006, 'order_date': '2025-11-02', 'region': 'South', 'product': 'Office-Lamp',  'quantity': 4, 'unit_price': 35,  'returned': 0},
    {'order_id': 1007, 'order_date': '2025-11-07', 'region': 'West',  'product': 'Tech-Keyboard','quantity': 2, 'unit_price': 60,  'returned': 0},
    {'order_id': 1008, 'order_date': '2025-11-09', 'region': 'East',  'product': 'Office-Desk',  'quantity': 1, 'unit_price': 260, 'returned': 1}
]
raw_df = pd.DataFrame(raw_data)
raw_df.head()

In [ ]:
# Transform to tidy format
tidy_df = raw_df.copy()
tidy_df['order_date'] = pd.to_datetime(tidy_df['order_date'])
tidy_df[['category', 'item']] = tidy_df['product'].str.split('-', n=1, expand=True)
tidy_df['gross_sales'] = tidy_df['quantity'] * tidy_df['unit_price']
tidy_df['net_sales'] = tidy_df['gross_sales'].where(tidy_df['returned'] == 0, 0)
tidy_df['month'] = tidy_df['order_date'].dt.to_period('M').astype(str)
tidy_df = tidy_df[['order_id', 'order_date', 'month', 'region', 'category', 'item', 'quantity', 'unit_price', 'returned', 'net_sales']]

print('Tidy table preview:')
tidy_df

In [ ]:
# Interesting stat
region_sales = tidy_df.groupby('region', as_index=False)['net_sales'].sum().sort_values('net_sales', ascending=False)
top_region = region_sales.iloc[0]
print(f"Top region by net sales: {top_region['region']} (${top_region['net_sales']:.2f})")
region_sales

In [ ]:
# Graph: monthly net sales trend
monthly = tidy_df.groupby('month', as_index=False)['net_sales'].sum()

plt.figure(figsize=(7, 4))
ax = sns.lineplot(data=monthly, x='month', y='net_sales', marker='o', linewidth=2.5, color='#1f77b4')
ax.set_title('Monthly Net Sales (Draft Dataset)')
ax.set_xlabel('Month')
ax.set_ylabel('Net Sales ($)')
plt.tight_layout()
plt.show()

## ETL Notes and Challenges
- Product strings had combined information (category + item), so they were split into separate tidy columns.
- Return records needed a business rule to avoid overstating revenue; this draft sets returned transactions to zero net sales.
- Dates were parsed and standardized to monthly periods for trend analysis.

In the final project, I will apply the same workflow to the chosen real dataset, add more robust validation checks, and extend the analysis with deeper comparisons (category-level seasonality, return-rate risk, and operational recommendations).